In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [100]:
X = df[['age','bmi','children','smoker','sex','region']]
y = df['charges']
X.head()

,age,bmi,children,smoker,sex,region
0,19,27.900,0,yes,female,southwest
1,18,33.770,1,no,male,southeast
2,28,33.000,3,no,male,southeast
3,33,22.705,0,no,male,northwest
4,32,28.880,0,no,male,northwest


In [4]:
# X.smoker = X.smoker.apply(lambda x: int(x=="yes"))
X.smoker.dtype == 'O'

True

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
print(X_train.shape)

(1070, 6)


In [124]:
X.dtypes

age           int64
bmi         float64
children      int64
smoker       object
sex          object
region       object
dtype: object

In [7]:
from sklearn.linear_model import LinearRegression, Lasso

cls = Lasso()
cls.fit(X_train, y_train)

Lasso()

In [8]:
y_pred = cls.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error
import numpy as np

err = np.sqrt(mean_squared_error(y_pred, y_test))

In [10]:
print(err)

5936.878148718208


In [11]:
from sklearn.svm import SVR
cls1 = SVR()
cls1.fit(X_train, y_train)

SVR()

In [12]:
y_pred1 = cls1.predict(X_test)

In [13]:
err = np.sqrt(mean_squared_error(y_pred1, y_test))

In [14]:
print(err)

12569.63049948133


In [24]:
from sklearn.ensemble import RandomForestRegressor
cls2 = RandomForestRegressor()
cls2.fit(X_train, y_train)
y_pred2 = cls2.predict(X_test)
err = np.sqrt(mean_squared_error(y_pred2, y_test))
print(err)

4849.426715700624


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader

C:\Users\E30794\.conda\envs\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [86]:
class pdDataset(Dataset):
    def __init__(self, df, lbl, transform = None, train=True):
        super().__init__()
        self.df = df.copy()
        self.lbl = lbl.copy()
        if transform is not None:
            for k in self.df.columns.values:
                if k in transforms:
                    if train:
                        self.df[k] = transform[k].fit_transform(self.df[k].values.reshape((-1,1))).squeeze()
                    else:
                        self.df[k] = transform[k].transform(self.df[k].values.reshape((-1,1))).squeeze()
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,index):
        x = self.df.iloc[index]#.values.astype(np.float32)
        y = torch.tensor(self.lbl.iloc[index])
        
        return x, y
    
def collate_fn(batch):
    x, y = batch[0]
    keys = x.keys()
    # print(keys)
    xs, ys =  zip(*[(x, y) for x, y in batch])
    
    X = {k: torch.stack([torch.tensor(x[k]) for x in xs], dim=0).reshape(-1,1) for k in keys}
    Y = torch.stack(ys, dim=0).long()
        
    return X, Y
    

In [125]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
transforms = dict()
for k in X.columns.values:
    if X[k].dtype == 'O':
        transforms[k] = LabelEncoder()
    else:
        transforms[k] = StandardScaler()

In [134]:
train_dataset = pdDataset(X_train, y_train, transform=transforms, train=True)
test_dataset = pdDataset(X_test, y_test, transform=transforms, train=False)
train_dataloader = DataLoader(train_dataset, batch_size=64, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

C:\Users\E30794\.conda\envs\python37\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\E30794\.conda\envs\python37\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [135]:
class fcmodel(nn.Module):
    def __init__(self, dim_in, hidden_size=64, n_layers = 4):
        super().__init__()
        layers = [nn.Linear(dim_in, hidden_size), nn.ReLU()]
        for _ in range(n_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, 1))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

In [150]:
from einops import repeat, rearrange
class TransModel(nn.Module):
    def __init__(self, fields_in, dim_in=1, dim=32, n_layers = 4, hidden_size=32):
        super().__init__()
        self.embeds = nn.ModuleDict(
            {
                col+'_': nn.Sequential(nn.Linear(dim_in, dim), nn.ReLU(), nn.Linear(dim, dim), nn.ReLU())
                for col in fields_in
            }
        )
        
        self.field_emb = nn.ParameterDict({
            col+'_': nn.Parameter(torch.randn(1, dim))
            for col in fields_in
        })
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=8) #, batch_first=True)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)
        
        # self.cls = nn.Parameter(torch.randn(1,1, dim))
        # self.mlp_head = nn.Sequential(nn.LayerNorm(dim), nn.Linear(dim, 1))
        layers = [nn.Linear(dim*len(fields_in), hidden_size), nn.ReLU()]
        for _ in range(n_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, 1))
        self.mlp_head = nn.Sequential(*layers)
        
    def forward(self, x):
        embed = []
        # print(x)
        for k in x:
            bs = x[k].shape[0]
            feat = self.embeds[k+'_'](x[k].float())
            field_enc = repeat(self.field_emb[k+'_'], '() d -> b d', b=bs)
            embed.append(feat + field_enc)
        
        x = torch.stack(embed, dim=1) # (b, fields, dim)
        x = rearrange(x, 'b n d -> n b d')
        
        # cls = repeat(self.cls, 'n () d -> n b d', b=bs)
        # x = torch.cat((cls, x), dim=0) # (fields + 1, b, dim)
        
        x = self.transformer(x)
        x = rearrange(x, 'n b d -> b (n d)') # concatenate
        x = self.mlp_head(x)
        
        return x

In [154]:
from tqdm import tqdm
print(X_train.columns.values)
model = TransModel(list(X_train.columns.values), n_layers=4)
print(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
with tqdm(range(100)) as te:
    for epoch in te:
        for batch in train_dataloader:
            optimizer.zero_grad()
            x, y = batch
            y_pred = model(x) #.float())
            loss = criterion(y_pred.squeeze(1), y.float())
            loss.backward()
            optimizer.step()
        te.set_postfix(loss=loss.item())

['age' 'bmi' 'children' 'smoker' 'sex' 'region']
TransModel(
  (embeds): ModuleDict(
    (age_): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
    )
    (bmi_): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
    )
    (children_): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
    )
    (smoker_): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): ReLU()
    )
    (sex_): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=3

100%|██████████████████████████████████████████████████████████████████| 100/100 [01:55<00:00,  1.16s/it, loss=1.19e+7]


In [157]:
y_pred = []
ytest = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        x, y = batch
        pred = model(x).squeeze(1)
        y_pred.append(pred)
        ytest.append(y)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 45.51it/s]


In [158]:
import numpy as np
from sklearn.metrics import mean_squared_error

y_pred_ml = torch.cat(y_pred, 0).to('cpu').numpy()
err = np.sqrt(mean_squared_error(y_pred_ml, y_test))
print(err)

4357.519827477643


In [159]:
model.field_emb

ParameterDict(
    (age_): Parameter containing: [torch.FloatTensor of size 1x32]
    (bmi_): Parameter containing: [torch.FloatTensor of size 1x32]
    (children_): Parameter containing: [torch.FloatTensor of size 1x32]
    (region_): Parameter containing: [torch.FloatTensor of size 1x32]
    (sex_): Parameter containing: [torch.FloatTensor of size 1x32]
    (smoker_): Parameter containing: [torch.FloatTensor of size 1x32]
)